# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iwanai,42.9744,140.5089,70.99,92,4,4.41,JP,Fri Aug 11 15:55:25 2023
1,1,el bauga,18.2620,33.9081,97.95,24,23,5.66,SD,Fri Aug 11 15:55:25 2023
2,2,olonkinbyen,70.9221,-8.7187,44.83,99,100,5.19,SJ,Fri Aug 11 15:55:26 2023
3,3,adamstown,-25.0660,-130.1015,65.61,51,11,19.42,PN,Fri Aug 11 15:55:26 2023
4,4,sosnovka,56.2550,51.2842,62.26,66,2,1.28,RU,Fri Aug 11 15:55:27 2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [2]:
#%%capture --no-display
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    color = "City",
    geo = True,
    tiles = "OSM",
    alpha=1,
    frame_width = 700,
    frame_height = 500,
    scale = 1    
)
map_plot_1

NameError: name 'city_data_df' is not defined

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
#locate rows using .loc that meet criteria for ideal city
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"]>30) & 
                            (city_data_df["Max Temp"]<95) &
                            (city_data_df["Wind Speed"]<4.5) & 
                            (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna(how='any')

# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,ilhabela,-23.7781,-45.3581,73.76,81,0,2.91,BR,Fri Aug 11 15:55:42 2023
78,78,port elizabeth,-33.9180,25.5701,55.69,76,0,2.30,ZA,Fri Aug 11 15:50:51 2023
112,112,svetlyy,54.6750,20.1347,56.97,72,0,3.53,RU,Fri Aug 11 15:56:01 2023
177,177,mermoz boabab,14.7065,-17.4758,83.98,74,0,2.30,SN,Fri Aug 11 15:56:28 2023
203,203,knittelfeld,47.2167,14.8167,59.47,82,0,2.30,AT,Fri Aug 11 15:56:37 2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
51,ilhabela,BR,-23.7781,-45.3581,81,
78,port elizabeth,ZA,-33.9180,25.5701,76,
112,svetlyy,RU,54.6750,20.1347,72,
177,mermoz boabab,SN,14.7065,-17.4758,74,
203,knittelfeld,AT,47.2167,14.8167,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
#Set the Radius as <10,000 from Lat & Lng

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "api_key":geoapify_key,
    "limit":30
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Lat=row["Lat"]
    Long=row["Lng"]
    
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Long},{Lat},{radius}"
    params["bias"] = f"proximity:{Long},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ilhabela - nearest hotel: Vila Kebaya
port elizabeth - nearest hotel: Waterford Hotel
svetlyy - nearest hotel: Гостиница Спортивная
mermoz boabab - nearest hotel: Le Shéhérazade
knittelfeld - nearest hotel: Zeilinger Villa
khandyga - nearest hotel: Апельсин
vadso - nearest hotel: Scandic Vadsø
khash - nearest hotel: No hotel found
muan - nearest hotel: 동남호텔
colonia - nearest hotel: Wasserturm Hotel Cologne
kalemie - nearest hotel: Hotel du Lac
mountain home - nearest hotel: Best Western Foothills Inn
fantanele - nearest hotel: Frunzei garsoniere
buka - nearest hotel: No hotel found
ypsonas - nearest hotel: Club Aphrodite
saint-francois - nearest hotel: Chez Lily
dul'durga - nearest hotel: No hotel found
mulayjah - nearest hotel: No hotel found
kabansk - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
kamina - nearest hotel: Hôtel DG


,City,Country,Lat,Lng,Humidity,Hotel Name
51,ilhabela,BR,-23.7781,-45.3581,81,Vila Kebaya
78,port elizabeth,ZA,-33.9180,25.5701,76,Waterford Hotel
112,svetlyy,RU,54.6750,20.1347,72,Гостиница Спортивная
177,mermoz boabab,SN,14.7065,-17.4758,74,Le Shéhérazade
203,knittelfeld,AT,47.2167,14.8167,82,Zeilinger Villa
241,khandyga,RU,62.6667,135.6000,63,Апельсин
248,vadso,NO,70.0744,29.7487,77,Scandic Vadsø
252,khash,IR,28.2211,61.2158,18,No hotel found
278,muan,KR,34.9897,126.4714,94,동남호텔
287,colonia,DE,50.9333,6.9500,70,Wasserturm Hotel Cologne


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE